# 시가총액 데이터 분석 I

> `강의_비즈데이터분석_2204/02-데이터처리/`pd-Cookbook3(2020시가총액파일).ipynb

1. pandas if conditions
1. 새열 추가
1. pandas where()
1. np.where 결합
1. loc,iloc 인덱싱
1. loc if-then
1. applymap
1. Grouping
1. Pivot

### 코랩: 한글 폰트 설치

1. 한글 폰트 설치

```sh
#폰트 설치
! apt install fonts-nanum*
```

```sh
! apt install fontconfig
```

2. 폰트 캐시 생성

```sh
#폰트 캐시 생성
!fc-cache -fv
```

3. matplotlib에 남아있는 font 캐시 삭제

```sh
# matplotlib에 남아있는 font 캐시 삭제
!rm -rf ~/.cache/matplotlib/*
```

4. Runtime 재실행

5. matplotlib 에서 Nanum 한글 폰트 검색 확인

```python
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 폰트 목록에서 폰트 찾기
for font in fm.fontManager.ttflist:
    if 'Nanum' in font.name:
        print(font.name, font.fname)
```

### 필요 라이브러리

In [ ]:
import numpy as np
import pandas as pd
import pickle

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
%matplotlib inline

In [ ]:
# 전역 폰트 설정 사용
# font_path = 'C:/Windows/Fonts/NanumGothic.ttf'
# font_path = '/Users/qkboo/Library/Fonts/NanumGothic.otf'
font_path = "/usr/share/fonts/truetype/nanum/NanumGothic.ttf"

fontname = fm.FontProperties(fname=font_path, size=18).get_name()  # 폰트 패밀리 이름!
# plt.rc('font', family=fontname)
plt.rcParams["font.family"] = fontname
plt.rcParams["font.size"] = 12

# matplotlib 설정
plt.rcParams["axes.grid"] = True
plt.rcParams["figure.figsize"] = (10,3)
plt.rcParams["axes.formatter.useoffset"] = False
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams["axes.formatter.limits"] = -10000, 10000

plt.title('한글 타이틀...')

# 2. FinanceDataReader 활용

In [ ]:
! pip install finance-datareader

In [ ]:
import FinanceDataReader as fdr

In [ ]:
pd.options.display.float_format = '{:.1f}'.format

# df2021 = pd.read_csv('../data/stocks/marcap-2021.csv.gz')
# df2021.head(3)

In [ ]:
df005930 = fdr.DataReader('005930', '2000-01-01', '2021-12-31')

자료의 전체 정보를 보자

In [ ]:
df005930.info()

## if conditions

컬럼에 대해 if 처리

In [ ]:
# 6만전자
df6 = df005930.loc[df005930.Close <60000]
df6.head(3)

In [ ]:
df6.index.min(), df6.index.max()

In [ ]:
# 종목중 종가가 80000 이상 경우만 추출
df8 = df005930.loc[df005930.Close >= 80000]
df8.index.min(), df.index.max()

6만전자에서 8만 전자까지 기간

In [ ]:
df6.index.max() - df8.index.min()

조건에 대한 결과는 여러 연산을 연결 할 수 있다.

In [ ]:
# 종가가 80000 미만인 날짜만 출력.
df005930.loc[df005930.Close <= 80000]

다중 조건도 가능

In [ ]:
# 종가가 80000원 미만이고 5/1일 이후만 출력!
df005930.loc[(df005930.Close <= 80000) & (df005930.Date >= '2020-05-01')].head(3)

### 연습) 5/1일 이전 10일 시가와 종가를 그래프로 그려보자

In [ ]:
df005930.loc[(df005930.index < '2021-05-01')][['Close']].plot()
# df005930.loc[(df005930.Date < '2021-05-01')][['Date','Close']].plot(x='Date')

In [ ]:
# 여러 종속변수를 지정
# df005930.loc[(df005930.Date < '2021-05-01')].plot(x='Date', y=['Open', 'Close'])
df005930.loc[(df005930.index < '2021-05-01')].plot( y=['Open', 'Close'])

10일 이전 데이터

In [ ]:
df005930.loc[(df005930.Date < '2021-05-01')][-10:].plot(x='Date', y=['Open', 'Close'])

## `pd.where()` 기본

### mask 로 사용: `pd.where()`


In [ ]:
dfmask = df005930['Close'] > 80000

In [ ]:
dfmask.info()

In [ ]:
df005930.where(dfmask)

In [ ]:
df005930.where(dfmask != np.NaN)

In [ ]:
df005930.where( (df005930['Close'] > 80000) != np.NaN )

## if-then : `pd.where()`

In [ ]:
df005930.where( (df005930['Close'] > 80000), 'Oppps' )

##  if-then-else: `np.where()`

if-then-else 는 `numpy`의 where() 와 함께 가능하다.

```python
np.where(condition, [x, y])
```

In [ ]:
# 종가가 5만 이상이면 5만전자, 아니면 4만전자
dfcopy = df005930.copy()
dfcopy['Nickname'] = np.where(dfcopy.Close > 80000, "8만전자", "7만전자")
dfcopy.loc[dfcopy.Nickname == '7만전자'].head(5)

## 조건 결합


In [ ]:
dfcopy = df005930.copy()
# 프레임을 등락율이 0.1 이상인 프레임으로 나눈다.
dfcopy[dfcopy.Change > 0.1][:5]

하나 혹은 다중 컬럼 조건은 `&`, `|` 로 묶어서 기준을 정하고, 나누어진 프레임 결과에서 보고자 하는 컬럼이름을 지정할 수 있다. 반환 컬럼이 하나면 Series 객체를 반환하고, 여러 컬럼을 지정하면 DataFrame 을 반환한다.

In [ ]:
# AND 조건
# 시가 7만 이상, 고가 7만 이상인 날짜의 종가를 출력
dfcopy.loc[(dfcopy.Open > 70000) & (dfcopy.High > 70000), "Close"][:4]

In [ ]:
# 여러 컬럼을 지정하면 DataFrame 객체를 반환한다.
dfcopy.loc[(dfcopy.Open > 70000) & (dfcopy.High > 70000), ["Close"]][:4]

In [ ]:
# OR 조건
dfcopy.loc[(dfcopy.Open > 70000) | (dfcopy.Low < 65000), ["Close"]][:4]

앞의 예제 처럼, 조건 일치한 결과에 대해서 컬럼을 수정할 수 있다.

In [ ]:
dfcopy.loc[(dfcopy.Open > 70000) | (dfcopy.Low < 65000), ["Changes"]] = 'HIGH'
dfcopy.loc[dfcopy.Changes == 'HIGH'][:5]

### `argsort()` 를 사용해 지정한 값에 근접한 데이터를 선택한다.

 - https://stackoverflow.com/questions/17758023/return-rows-in-a-dataframe-closest-to-a-user-defined-number

기준으로 바이너리 연산자 사용해서 대상을 줄일 수 도 있다(매스킹의 일부), 이런 기준들을 하나로 묶어 사용할 수 있다.

In [ ]:
crit1 = dfcopy.Close <= 50000
crit2 = (dfcopy.Open > 50000) & (dfcopy.Close < 60000 )
crit3 = dfcopy.High > 60000
allcrits = crit1 & crit2 & crit3

In [ ]:
dfcopy[crit2][:5]

In [ ]:
dfcopy[allcrits]

혹은 동적으로 기준을 리스트로 두고 `functools.reduce()` 에 저굥할수 있다.

In [ ]:
import functools

critlist = [crit1, crit2, crit3]
allcrits = functools.reduce(lambda x, y: x & y, critlist)
dfcopy[allcrits]

## loc, iloc 인덱싱

 - 전통적인 행 라벨과 값 상태를 이용할 수 있다.
 - `.loc[]` 를 사용해 라벨 지향과 `.iloc[]` 로 위치 값 기반

In [ ]:
# 종가가 5만원 미만이면서 변동폭이 2% 이상인 것
df005930[ (df005930.Close <= 50000) & (df005930.ChagesRatio > 2.0)]

행 라벨과 값 상태 양쪽을 사용할 수 있다.

In [ ]:
df005930[(df005930.Close <= 50000) & (df005930.index.isin([127182, 131872]))]

슬라이싱은 `.loc[]` 로 레이블 지향과 `.iloc[]` 로 인덱스 위치 슬라이싱을 사용할 때는 ,두 가지 명시적인 슬라이싱 방법과  세 번째 일반 슬라이싱 방법이 있다.

1. 위치 지향(Python 슬라이싱 스타일: 끝 제외)
2. 레이블 지향(비-파이톤 슬라이싱 스타일: 끝 포함)
3. 일반(슬라이싱 스타일 : 슬라이스에 레이블 또는 위치가 포함되어 있는지 여부에 따라 다름)

1. 위치 지향으로 인덱스 값을 기반으로 한다.


In [ ]:
# 인덱스에서 005930~006000 을 찾는다.
df005930.set_index(["Code", "Name"]).loc["005930":"006000"][:4]

In [ ]:
# 2. 일반 슬라이싱 스타일: 인덱스 위치 기반
df005930[2:5]

In [ ]:
# .iloc[] 인덱스 위치 기반 - 인덱스 위치 값이다.
df005930.iloc[2:5]

인덱스를 이름으로 변경하고, 위치 값 [2:5] 을 찾아보자,

In [ ]:
# 인덱스를 변경하면 위치값이 다르므로 결과가 다르다.
df005930.set_index(['Name']).iloc[2:5]

In [ ]:
# 인덱스 라벨로 2~5 라는 라벨이 없으므로 결과가 없다.
df005930.loc[2:5]

In [ ]:
# 인덱스를 날짜로 바꾸고 라벨로 슬라이싱을 해보자.
df005930.set_index(['Date'], inplace=True)
df005930.head(3)

In [ ]:
# 날짜형식으로 인덱스 범위를 지정.
df005930.loc["2020-02-03":"2020-02-06"]

In [ ]:
# 인덱스가 날짜 상태에서 위치 값으로 슬라이싱
df005930.iloc[1:4]

## `loc` 조건 if-then

컬럼에 대해 if 조건이 맞으면 대상 컬럼에 then 처리. 

```python
df.loc[ IF조건, "THEN"] = value
```

예를 들어 종가가 50000원 밑이면 Low 컬럼에 True 값을 주어 보자.

In [ ]:
df2 = df005930.copy()
df2.loc[df2.Close < 80000, "FOUR"] = True
df2.head(3)

In [ ]:
# 종가 80000 아래인 날짜
df2.loc[df2.FOUR == True]['Date'].unique()

if-then은 여러 컬럼에 대해서도 적용이 가능하다

In [ ]:
df2 = df005930.copy()

df2.loc[df2.Close > 50000, ['FOUR','SIX']] = (False,True)
df2.loc[df2.SIX == True].shape

### 보수 연산 `~` 사용

반대 연산자 `~` 를 사용해 매스크의 보수를 얻는다.

In [ ]:
# 50000원 이하이고 등락률이 2% 이상이 아닌 모든것
df005930[~((df005930.Close < 50000) & (df005930.ChagesRatio > 2.0))][:5]

# 참고

FinanceDataReader 사용

 - 설치 및 사용: https://github.com/financedata-org/FinanceDataReader
 - 사용 설명서: https://financedata.github.io/posts/finance-data-reader-users-guide.html
